**1. Libraries**

In [1]:
from requests_html import HTMLSession
import pandas as pd
import re, os, urllib, shutil
import pandas as pd
import sqlite3
import unidecode

**2. Connecting**

In [2]:
session = HTMLSession()

In [3]:
url = 'https://www.colombia.com/colombia-info/departamentos/'

In [4]:
response = session.get(url)

**3. Scrapping**

In [5]:
caja_informacion = response.html.find('div[class = caja-informacion]')

In [6]:
def replace_all(text, dic):
    for i, j in dic.iteritems():
        text = text.replace(i, j)
    return text

In [7]:
link_city = "a[href ^= '/colombia-info/departamentos/region-"
con = sqlite3.connect('Colombia_Demographic_data.sqlite', timeout=30)
cur = con.cursor()
for caja in caja_informacion:
    try:
        region = caja.find("a[href ^= '/colombia-info/departamentos/']")[0].text.capitalize()
        if len(region)>16:
            region = region[13:]
            region = unidecode.unidecode(region)
            condition = "".join([link_city, region,"/']"])
            region = region.capitalize()
            cities = caja.find(condition)

            
        else:
            region = region[7:]
            region = unidecode.unidecode(region)
            condition = "".join([link_city, region,"/']"])
            region = region.capitalize()
            cities = caja.find(condition)

        


        insert_region = 'INSERT INTO Region(name) VALUES'
        insert_region += '('
        insert_region += '"'+str(region) + '"'+'),('
        insert_region = insert_region[:-2] +';'
        cur.execute(insert_region)
        print('Region:', region)
        
        for city in cities[1:]:
            update_department = 'UPDATE Departamento SET  region_id ='
            try:
                city2 = city.text.capitalize()
                select_query = 'SELECT id FROM '
                select_query += 'Region' + ' WHERE name = ? ;'
                cur.execute(select_query, (region, ))
                region_id = cur.fetchone()[0]
                if city2 == 'San andrés y providencia':
                    city2 = "Archipiélago de san andrés. providencia y santa catalina"
                update_department += '"'+str(region_id) + '"'+' WHERE Departamento.name = ?;'
                cur.execute(update_department, (city2, ))

                
                con.commit()
                print("               ",city2)
            except:
                continue
            
    except: 
        continue
        
# Updating Bogotá
select_query = 'SELECT id FROM Region WHERE name = "Andina" ;'
cur.execute(select_query)
region_id = cur.fetchone()[0]
update_department = 'UPDATE Departamento SET  region_id =' + str(region_id) +' WHERE Departamento.name = "Bogotá. d.c.";'
cur.execute(update_department)
con.close()

Region: Andina
                Antioquia
                Boyacá
                Caldas
                Cundinamarca
                Huila
                Norte de santander
                Quindío
                Risaralda
                Santander
                Tolima
Region: Amazonica
                Amazonas
                Caquetá
                Guainía
                Guaviare
                Putumayo
                Vaupés
Region: Caribe
                Atlántico
                Bolívar
                Cesar
                Córdoba
                La guajira
                Magdalena
                Archipiélago de san andrés. providencia y santa catalina
                Sucre
Region: Pacifica
                Cauca
                Chocó
                Nariño
                Valle del cauca
Region: Orinoquia
                Arauca
                Casanare
                Meta
                Vichada
